In [29]:
import pandas as pd
import math
import ast

In [40]:
note2number = {
    'B#': 0,
    'C': 0,
    'C#': 1,
    'Db': 1,
    'D': 2,
    'D#': 3,
    'Eb': 3,
    'E': 4,
    'Fb': 4,
    'E#': 5,
    'F': 5,
    'F#': 6,
    'Gb': 6,
    'G': 7,
    'G#': 8,
    'Ab': 8,
    'A': 9,
    'A#': 10,
    'Bb': 10,
    'B': 11,
    'Cb': 11
}

def key2number(key):
    tonic = 0
    tonality = ''
    if len(key) > 2: # e.g. C#m
        tonality = 'min'
        tonic = note2number[key[:2]]
    elif len(key) == 2: # e.g. Dm or Db
        if key in note2number: # e.g. Db
            tonality = 'maj'
            tonic = note2number[key]
        else: # e.g. Dm
            tonality = 'min'
            tonic = note2number[key[:1]]
    else: # e.g. C
        tonality = 'maj'
        tonic = note2number[key]
    return tonic, tonality

def chord2number(chord, tonic):
    copy = chord

    # convert root to number
    root = 0
    root_length = 0
    if copy[:2] in note2number: root_length = 2 # e.g. C#
    elif copy[:1] in note2number: root_length = 1 # e.g. C
    root = note2number[copy[:root_length]]
    root_relative = root - tonic + 12 if root - tonic < 0 else root - tonic
    copy = str(root_relative) + copy[root_length:]

    # check if slash chord
    slash_index = 0
    for index, char in enumerate(copy):
        if char == '/': slash_index = index
    
    # convert if slash chord
    if slash_index > 0:
        slash_root = 0
        slash_root_relative = 0
        slash_root = note2number[copy[slash_index + 1:]]
        slash_root_relative = slash_root - tonic + 12 if slash_root - tonic < 0 else slash_root - tonic
        copy = copy[:slash_index + 1] + str(slash_root_relative)

    return copy

In [28]:
import math
csv = pd.read_csv('./1980.csv')

for row in csv.itertuples(index = True):
    tonic, tonality = key2number(row.key)
    converted_lines = []
    chords = ast.literal_eval(row.chords)
    for line in chords:
        converted_line = []
        for chord in line:
            # convert shit
    break

True
